<a href="https://colab.research.google.com/github/DeadPool-4422/Collab-GDSC/blob/main/Face-Detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Capture Image from Webcam
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=1.0):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

# Take a photo
image_path = take_photo()

# Step 2: Upload and Process the Image
import cv2
from google.colab.patches import cv2_imshow

# Load the image
img = cv2.imread(image_path)
cv2_imshow(img)  # Display the image

# Step 3: Facial Recognition
# You can use a pre-trained model like Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.1, 4)

# Draw rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 250, 0), 2)

# Display the output
cv2_imshow(img)


In [ ]:
!pip install mtcnn

In [ ]:
from mtcnn.mtcnn import MTCNN
import cv2
from google.colab.patches import cv2_imshow

# Step 1: Capture Image from Webcam
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=1.0):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

# Take a photo
image_path = take_photo()

# Step 2: Upload and Process the Image
import cv2
from google.colab.patches import cv2_imshow

# Load the image
img = cv2.imread(image_path)

# Initialize MTCNN
detector = MTCNN()

# Detect faces in the image
faces = detector.detect_faces(img)

# Draw rectangle around the faces
for face in faces:
    x, y, width, height = face['box']
    cv2.rectangle(img, (x, y), (x + width, y + height), (255, 255, 0), 2)

# Display the output
cv2_imshow(img)


In [ ]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 9.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=4985a3c7a51cd5b2d61d38d9fbb7d21deb23bf9d8190dbc22b822d92c2757af0
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [ ]:
from mtcnn.mtcnn import MTCNN
from google.colab import files
from IPython.display import display
import cv2
from google.colab.patches import cv2_imshow
from deepface import DeepFace
import numpy as np
from io import BytesIO
from PIL import Image as PIL_Image

# Function to upload an image
def upload_image():
    uploaded = files.upload()
    if not uploaded:
        return None, None

    for fn in uploaded.keys():
        return fn, uploaded[fn]  # return the file name and file data

# Use the function to upload an image
image_path, image_data = upload_image()

# Load the uploaded image for further processing
if image_path is not None:
    pil_image = PIL_Image.open(BytesIO(image_data))
    img = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    # Now img can be used with OpenCV functions
else:
    print("No file uploaded")
    # Ensure the rest of the code does not execute if no image is uploaded
    raise SystemExit

# Initialize MTCNN for face detection
detector = MTCNN()

# Detect faces in the image
faces = detector.detect_faces(img)

for i, face in enumerate(faces):
    x, y, width, height = face['box']
    face_img = img[y:y+height, x:x+width]

    try:
        # Analyze the emotion of the face
        predictions = DeepFace.analyze(face_img, actions=['emotion'], enforce_detection=False)

        # Check the type of predictions and proceed accordingly
        face_data = predictions if isinstance(predictions, dict) else predictions[0]

        # Draw rectangle around the face and put text of emotion
        cv2.rectangle(img, (x, y), (x + width, y + height), (255, 255, 0), 2)
        emotion_text = face_data['dominant_emotion'] if 'dominant_emotion' in face_data else "No emotion detected"
        cv2.putText(img, emotion_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    except Exception as e:
        print(f"Error in processing face: {e}")

# Display the output
cv2_imshow(img)
